In [1]:
#####
# 1.最新四半期決算の営業利益率は低い
# 2.最新四半期決算の売上高成長率が10%以上
# 3.最新四半期決算の対売上高営業利益成長率が20%以上
# 4.次期決算を、以下の方法で予測した場合において、営業利益率が前年同期比2倍以上になる
# --> 最新四半期決算の売上高成長率と対売上高営業利益成長率を、次期前年四半期決算に適用して次期決算を予想する

In [2]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import KessanPl, PricelistFig, PricelistPl, MeigaralistPl
from lib_dataprocess import ShikihoPl, FinancequotePl, print_finance_quote, PricelistFig, KessanFig, IndexPricelistPl
import polars as pl
from datetime import date
from dateutil.relativedelta import relativedelta

RevPl = PricelistPl("reviced_pricelist.parquet")
RawPL = PricelistPl("raw_pricelist.parquet")
MPL = MeigaralistPl()
SPL = ShikihoPl()

In [3]:
KPL = KessanPl()
KPL.with_columns_lastyear_settlement()
df1 = KPL.df.filter(pl.col("settlement_type")=="四")
df1 = df1.with_columns([
    pl.col("settlement_date").alias("key")
])

In [4]:
# para
valuation_date = date(2024, 8, 1)

KPL = KessanPl()
df2 = KPL.get_settlement_forcast_by_diff_growth_rate(valuation_date)
df2 = df2.filter(pl.col("settlement_type")=="四")
df2 = df2.with_columns([
    pl.col("nxt_settlement_date").alias("key")
])

In [5]:
KPL = KessanPl()
df3 = KPL.get_current_settlements(valuation_date)
df3 = df3.filter(pl.col("settlement_type")=="四")
df3 = df3.with_columns([
    pl.col("settlement_date").alias("key")
])

In [41]:
KPL = KessanPl()
df = KPL.get_settlement_updown_rate("四")
df = df.with_columns([
    pl.col("settlement_date").alias("key")
])
df4 = df.select(["code", "key", "updown_rate", "nh_updown_rate"])
# df4.filter(pl.col("code")==9022)

code,key,updown_rate,nh_updown_rate
i64,date,f64,f64
9022,2018-12-31,10.1,-1.03
9022,2019-03-31,4.2,6.4
9022,2019-06-30,-10.9,-2.89
9022,2019-09-30,3.2,6.62
9022,2019-12-31,-5.1,0.86
…,…,…,…
9022,2024-06-30,-0.3,0.09
9022,2024-09-30,-11.2,4.31
9022,2024-09-30,-9.6,4.31


In [7]:
# df1, df2, df3, df4をjoin&select
df = df2.join(df1, on=["code", "key"], how="left")
df21 = df

df = df21.with_columns([
    pl.col("settlement_date").alias("lst_settlement_date"),
    pl.col("announcement_date").alias("lst_announcement_date"),
    pl.col("settlement_type").alias("lst_settlement_type"),
    pl.col("sales").alias("lst_sales"),
    pl.col("operating_income").alias("lst_operating_income"),
    pl.col("ordinary_profit").alias("lst_ordinary_profit"),
    pl.col("final_profit").alias("lst_final_profit"),
    pl.col("quater").alias("lst_quater")
])
df = df.select([
    "code",	
    "lst_settlement_date",
    "lst_announcement_date",
    "lst_settlement_type",
    'lst_sales',
    'lst_operating_income',
    'lst_ordinary_profit',
    'lst_final_profit',
    "quater",
    "ly_settlement_date",
    "ly_sales",
    "ly_operating_income",
    "ly_ordinary_profit",
    "ly_final_profit",
    "nxt_settlement_date",
    "fcst_dgr_sales",
    "fcst_dgr_operating_income",
    "fcst_dgr_ordinary_profit",	
    "fcst_dgr_final_profit",
    "key"
])
df = df.join(df3, on=["code", "key"], how="left")
df = df.drop_nulls()

df123 = df

In [8]:
# ly_fcstの売上高、営業利益成長率を
# 列名ly_fcst_sales_gr, ly_fcst_operating_income_grで追加
df = df123.with_columns([
    (pl.lit(100) * pl.col("lst_operating_income") / pl.col("lst_sales")).round(2).alias("lst_operating_income_pr"),
    (pl.lit(100) * (pl.col("fcst_dgr_sales") - pl.col("ly_sales")) / pl.col("ly_sales")).round(2).alias("ly_fcst_sales_gr"),
    (pl.lit(100) * (pl.col("fcst_dgr_operating_income") - pl.col("ly_operating_income")) / pl.col("ly_operating_income")).round(2).alias("ly_fcst_operating_income_gr"),
])

df = df.join(df4, on=["code", "key"], how="left")
df = df.drop_nulls()

scols = [
    "code",
    "lst_settlement_date",
    #lst_announcement_date,
    #lst_settlement_type,
    "lst_sales",
    "lst_operating_income",
    "lst_ordinary_profit",
    "lst_final_profit",
    "quater",
    #ly_settlement_date,
    "ly_sales",
    "ly_operating_income",
    #ly_ordinary_profit,
    #ly_final_profit,
    #nxt_settlement_date,
    "fcst_dgr_sales",
    "fcst_dgr_operating_income",
    "fcst_dgr_ordinary_profit",
    "fcst_dgr_final_profit",
    #key,
    #settlement_date,
    #settlement_type,
    #announcement_date,
    "sales",
    "operating_income",
    "ordinary_profit",
    "final_profit",
    #reviced_eps,
    #dividend,
    #quater_right,
    "lst_operating_income_pr",
    "ly_fcst_sales_gr",
    "ly_fcst_operating_income_gr",
    "updown_rate",
    "nh_updown_rate"
]
df = df.select(scols)
codes = df["code"].unique()
df = df.sort(by=["ly_fcst_operating_income_gr"], descending=[True])
original_df = df
original_df

code,lst_settlement_date,lst_sales,lst_operating_income,lst_ordinary_profit,lst_final_profit,quater,ly_sales,ly_operating_income,fcst_dgr_sales,fcst_dgr_operating_income,fcst_dgr_ordinary_profit,fcst_dgr_final_profit,sales,operating_income,ordinary_profit,final_profit,lst_operating_income_pr,ly_fcst_sales_gr,ly_fcst_operating_income_gr,updown_rate,nh_updown_rate
i64,date,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64
4406,2024-03-31,8783,456,476,303,4,7952,1,9986,1227,1205,896,8194,203,281,236,5.19,25.58,122600.0,4.8,-7.12
9438,2024-03-31,6892,617,726,928,2,6500,1,7271,1227,1206,1633,7019,700,845,685,8.95,11.86,122600.0,3.5,-11.05
4418,2024-03-31,5531,41,28,8,3,452,1,55022,833,721,651,5292,4,-12,-213,0.74,12073.01,83200.0,-13.7,-4.39
6145,2024-03-31,11465,2173,2165,1395,4,4195,9,16383,3727,3852,2320,4319,-611,-484,-345,18.95,290.54,41311.11,-27.0,-7.12
1789,2024-03-31,2303,341,342,218,2,1773,1,1916,411,407,250,1675,-37,-43,-33,14.81,8.07,41000.0,-26.8,-7.9
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
5967,2024-05-31,2568,684,702,539,4,1305,-5,2917,922,956,779,1475,35,10,-20,26.64,123.52,-18540.0,17.5,-3.89
1961,2024-03-31,72309,5588,5770,4090,4,40727,-18,80263,7083,7472,4527,48527,1540,2087,1345,7.73,97.08,-39450.0,2.6,-7.12
7494,2024-03-31,19278,1143,1145,653,2,16144,-2,19104,1514,1457,811,14560,-599,-696,-829,5.93,18.33,-75800.0,-28.4,-5.86


In [25]:
df = original_df
df = df.group_by(["code"]).agg([
    pl.col("lst_settlement_date").count()
])
df = df.filter(pl.col("lst_settlement_date")!=1)
df

code,lst_settlement_date
i64,u32
9022,4


In [42]:
df = original_df
df = df.filter(pl.col("code")==9022)
df

code,lst_settlement_date,lst_sales,lst_operating_income,lst_ordinary_profit,lst_final_profit,quater,ly_sales,ly_operating_income,fcst_dgr_sales,fcst_dgr_operating_income,fcst_dgr_ordinary_profit,fcst_dgr_final_profit,sales,operating_income,ordinary_profit,final_profit,lst_operating_income_pr,ly_fcst_sales_gr,ly_fcst_operating_income_gr,updown_rate,nh_updown_rate
i64,date,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64
9022,2024-06-30,435290,184327,171278,119786,1,422454,165574,479690,226551,218031,152509,438575,180882,164189,113861,42.35,13.55,36.83,-11.2,4.31
9022,2024-06-30,435290,184327,171278,119786,1,422454,165574,479690,226551,218031,152509,438575,180882,164189,113861,42.35,13.55,36.83,-9.6,4.31
9022,2024-06-30,435290,184327,171278,119786,1,422454,165574,479690,226551,218031,152509,438575,180882,164189,113861,42.35,13.55,36.83,-9.6,2.52
9022,2024-06-30,435290,184327,171278,119786,1,422454,165574,479690,226551,218031,152509,438575,180882,164189,113861,42.35,13.55,36.83,-11.2,2.52


In [9]:
######
###### filter
######

In [10]:
df = original_df
df.sort

<bound method DataFrame.sort of shape: (3_537, 22)
┌──────┬────────────┬───────────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ code ┆ lst_settle ┆ lst_sales ┆ lst_operat ┆ … ┆ ly_fcst_sa ┆ ly_fcst_op ┆ updown_ra ┆ nh_updown │
│ ---  ┆ ment_date  ┆ ---       ┆ ing_income ┆   ┆ les_gr     ┆ erating_in ┆ te        ┆ _rate     │
│ i64  ┆ ---        ┆ i64       ┆ ---        ┆   ┆ ---        ┆ come_gr    ┆ ---       ┆ ---       │
│      ┆ date       ┆           ┆ i64        ┆   ┆ f64        ┆ ---        ┆ f64       ┆ f64       │
│      ┆            ┆           ┆            ┆   ┆            ┆ f64        ┆           ┆           │
╞══════╪════════════╪═══════════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 4406 ┆ 2024-03-31 ┆ 8783      ┆ 456        ┆ … ┆ 25.58      ┆ 122600.0   ┆ 4.8       ┆ -7.12     │
│ 9438 ┆ 2024-03-31 ┆ 6892      ┆ 617        ┆ … ┆ 11.86      ┆ 122600.0   ┆ 3.5       ┆ -11.05    │
│ 4418 ┆ 2024-03-31 ┆ 5531      ┆ 41    

In [28]:
KPL = KessanPl()
df = KPL.df
df = df.filter(pl.col("code")==9022).filter(pl.col("settlement_date")==date(2024, 3, 31))

In [29]:
df

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
9022,2024-03-31,"""予""",2023-04-26,1566000,430000,357000,250000,1270.3,140.0,4
9022,2024-03-31,"""予""",2023-08-22,1566000,430000,357000,250000,1270.3,84.0,4
9022,2024-03-31,"""予""",2023-10-30,1627000,502000,433000,308000,313.0,28.0,4
9022,2024-03-31,"""予""",2024-01-30,1658000,533000,464000,330000,335.4,28.0,4
9022,2024-03-31,"""四""",2024-04-30,437390,104784,91831,65641,66.7,24.0,4
9022,2024-03-31,"""本""",2024-04-30,1710407,607381,546946,384411,390.7,29.0,4


In [38]:
df3.filter(pl.col("code")==9022).filter(pl.col("settlement_date")==date(2024, 6, 30))

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater,key
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64,date
